In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv(r'E:\Kaggle\Titanic\data\train.csv')
X, y = data.drop(["Survived"], axis=1), data["Survived"]

Check Data Completeness

In [5]:
print(X.count() / len(X))

PassengerId    1.000000
Pclass         1.000000
Name           1.000000
Sex            1.000000
Age            0.801347
SibSp          1.000000
Parch          1.000000
Ticket         1.000000
Fare           1.000000
Cabin          0.228956
Embarked       0.997755
dtype: float64


Missing value observed, skip these features for the time being, avail of them if necessary

In [6]:
X = X.drop(["Age", "Cabin", "Embarked"], axis=1)

In [7]:
X = X.drop(["Name", "Ticket"], axis=1)

In [8]:
X

,PassengerId,Pclass,Sex,SibSp,Parch,Fare
0,1,3,male,1,0,7.2500
1,2,1,female,1,0,71.2833
2,3,3,female,0,0,7.9250
3,4,1,female,1,0,53.1000
4,5,3,male,0,0,8.0500
...,...,...,...,...,...,...
886,887,2,male,0,0,13.0000
887,888,1,female,0,0,30.0000
888,889,3,female,1,2,23.4500
889,890,1,male,0,0,30.0000


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X["Sex"] = le.fit_transform(X["Sex"])
X

,PassengerId,Pclass,Sex,SibSp,Parch,Fare
0,1,3,1,1,0,7.2500
1,2,1,0,1,0,71.2833
2,3,3,0,0,0,7.9250
3,4,1,0,1,0,53.1000
4,5,3,1,0,0,8.0500
...,...,...,...,...,...,...
886,887,2,1,0,0,13.0000
887,888,1,0,0,0,30.0000
888,889,3,0,1,2,23.4500
889,890,1,1,0,0,30.0000


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

RandomForestClassifier()

In [35]:
from sklearn.metrics import confusion_matrix, accuracy_score

rf_y_pred = rf_model.predict(x_test)
rf_cfm = confusion_matrix(y_test, rf_y_pred)
rf_acc = accuracy_score(y_test, rf_y_pred)

rf_cfm, rf_acc

(array([[90, 13],
        [22, 54]], dtype=int64),
 0.8044692737430168)

In [15]:
from sklearn import svm

svm_model = svm.SVC()
svm_model.fit(x_train, y_train)
svm_y_pred = svm_model.predict(x_test)

svm_cfm = confusion_matrix(y_test, svm_y_pred)
svm_acc = accuracy_score(y_test, svm_y_pred)

svm_cfm, svm_acc

(array([[99,  4],
        [65, 11]], dtype=int64),
 0.6145251396648045)

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lr_model = LogisticRegression(max_iter=2000)
lr_model.fit(x_train, y_train)
lr_y_pred = lr_model.predict(x_test)

lr_cfm, lr_acc = confusion_matrix(y_test, lr_y_pred), accuracy_score(y_test, lr_y_pred)
lr_cfm, lr_acc

(array([[88, 15],
        [22, 54]], dtype=int64),
 0.7932960893854749)

In [51]:
lda = LinearDiscriminantAnalysis()

x_train_lda = lda.fit_transform(x_train, y_train)
x_test_lda = lda.transform(x_test)

lr_model_lda = LogisticRegression(max_iter=2000)
lr_model_lda.fit(x_train_lda, y_train)
lr_y_pred_lda = lr_model_lda.predict(x_test_lda)

lr_lda_cfm, lr_lda_acc = confusion_matrix(y_test, lr_y_pred_lda), accuracy_score(y_test, lr_y_pred_lda)
lr_lda_cfm, lr_lda_acc

(array([[88, 15],
        [21, 55]], dtype=int64),
 0.7988826815642458)

In [49]:
def conclude(y_true, y_pred):
    print(confusion_matrix(y_test, y_pred), accuracy_score(y_test, y_pred))

In [50]:
# from sklearn.ensemble import GradientBoostingClassifier as XGBoost
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    learning_rate=0.01,
    n_estimators=10,
    max_depth=4,
    min_child_weight = 1,
    gamma=0.,
    subsample=1,
    colsample_btree=1,
    scale_pos_weight=1,
    random_state=27,
    slient = 0
)
xgb_model.fit(x_train, y_train)
xgb_y_pred = xgb_model.predict(x_test)

conclude(y_test, xgb_y_pred)

[12:20:43] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "colsample_btree", "slient" } are not used.

[[74 29]
 [15 61]] 0.7541899441340782


In [53]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(x_train)
print(pca.explained_variance_ratio_)

pca = PCA(n_components=2)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.fit_transform(x_test)

model = LogisticRegression()
model.fit(x_train_pca, y_train)
y_pred_pca = model.predict(x_test_pca)

conclude(y_test, y_pred_pca)

[9.61519656e-01 3.84453957e-02 1.88806946e-05]
[[91 12]
 [61 15]] 0.5921787709497207


In [59]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(64, 32, 16), activation='relu')
mlp.fit(x_train, y_train)
print(mlp.score(x_test, y_test))

mlp_y_pred = mlp.predict(x_test)
conclude(y_test, mlp_y_pred)

0.6815642458100558
[[70 33]
 [24 52]] 0.6815642458100558


No algorithm performed well